In [ ]:
from PyPDF2 import PdfReader
import re
from first import first
import pandas as pd

In [ ]:
filepath = "/content/HHP_Dec23_Topline"

In [ ]:
def page_to_question(page):
  lines = [line.strip() for line in page.extract_text().splitlines() if not line.isspace()]
  total_index = lines.index("Total")

  (symbol, text) = lines[6].split(maxsplit=1)
  base = lines[total_index - 1].split()[1]
  unweighted = lines[total_index + 1].split()[-1]
  weighted = lines[total_index + 2].split()[-1]

  option_lines = [re.sub("\s+\d+%", "", line) for line in lines[total_index + 3:] if " " in line]
  options = []
  for line in option_lines:
    if re.search('\s+[\d\-]', line):
      options.append(re.split("\s+(?=[\d\-])", line, maxsplit=1))
    else:
      options[-1][0] += " " + line
  options = dict((key, value) for key, value in options)

  return {
      "Symbol": symbol,
      "Text": text,
      "Base": base,
      "Unweighted": unweighted,
      "Weighted": weighted,
      "Options": options
  }

In [ ]:
reader = PdfReader(filepath + ".pdf")
questions = [page_to_question(page) for page in reader.pages[2:]]
df = pd.DataFrame.from_records(questions, columns=["Symbol", "Text", "Base", "Unweighted", "Weighted", "Options"])
df.to_csv(filepath + ".csv", index=False)